# This notebook is going to serve as my workflow for estimating NCF bathymetries using historical eHydro sureys and EO multispectral data. I will train a NN on eHydro bathy surveys to estimate waterway bathymetries using corrected surface reflectances
- get bathy surveys from https://services7.arcgis.com/n1YM8pTrFmm7L4hs/ArcGIS/rest/services/eHydro_Survey_Data/FeatureServer/query
- S2 imagery from GEE
- TF for NN

# Query bathy data

In [ ]:
import requests

# Define the URL for the ArcGIS REST service
url = "https://services7.arcgis.com/n1YM8pTrFmm7L4hs/ArcGIS/rest/services/eHydro_Survey_Data/FeatureServer/0/query"

# Define the parameters for the query (without geometry)
params = {
    'where': "1=1",  # No filter, retrieve all data
    'outFields': '*',  # Retrieve all fields
    'f': 'json',  # Output format
    'outSR': '4326',  # Output spatial reference (WGS 84 for Lat/Long)
}

# Send the request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()
    
    # Extract features (bathymetric survey data)
    features = data.get('features', [])
    
    # Iterate through features to extract bathymetric values and coordinates
    for feature in features:
        geometry = feature.get('geometry', {})
        attributes = feature.get('attributes', {})
        print(f"Geometry: {geometry}, Attributes: {attributes}")
else:
    print(f"Failed to retrieve data: {response.status_code}")

In [ ]:
params['resultOffset'] = 0
params['resultRecordCount'] = 1000  # Adjust depending on the number of records you want

while True:
    response = requests.get(url, params=params)
    data = response.json()
    features = data.get('features', [])
    
    if not features:
        break  # No more results
    
    # Process features
    for feature in features:
        geometry = feature.get('geometry', {})
        attributes = feature.get('attributes', {})
        print(f"Geometry: {geometry}, Attributes: {attributes}")
    
    # Increment offset for the next batch of records
    params['resultOffset'] += 1000


# Get EE imagery for each survey
- Sentinel-2 Surface Reflectance product, test multiple individual bands (red, blue, NIR may be best)
- At some point, if S2 results good, try Planet 3m-daily imagery